In [ ]:
cd ~/data/ciss/opus

In [ ]:
!which python

In [ ]:
fnames = !ls *map.cal.cub
fnames

In [ ]:
from pyciss import io

In [ ]:
cube = io.RingCube(fnames[0])

In [ ]:
for fname in fnames:
    print(fname)
    ringcube = io.RingCube(fname)
    print(ringcube.imagetime)
#     ringcube.imshow(save_ext='')
#     ringcube.imshow(data=ringcube.density_wave_subtracted, save_ext='wave_subtracted')

In [ ]:
%matplotlib nbagg

In [ ]:
ringcube.imshow(data=ringcube.density_wave_subtracted)

In [ ]:
import gdal
ds = gdal.Open('img.cal.map.cub')
print("proj:", ds.GetProjection())
print("GeoTrans:", ds.GetGeoTransform())
x, y = gdal.ApplyGeoTransform(ds.GetGeoTransform(), 0, 0)
print("X:{}, Y:{}".format(x, y))
import osr
srs = osr.SpatialReference(ds.GetProjection())
print(srs.ExportToPrettyWkt())
ct = osr.CoordinateTransformation(srs, srs.CloneGeogCS())
print("CoordTrans:", ct)
print(ct.TransformPoint(x, y))

In [ ]:
gdal.VersionInfo()

In [ ]:
try:
    from osgeo import gdal, osr
except ImportError:
    import gdal
    import osr

import sys

# =============================================================================
def Usage():
    print('')
    print('Read coordinate system and geotransformation matrix from input')
    print('file and report latitude/longitude coordinates for the center')
    print('of the specified pixel.')
    print('')
    print('Usage: tolatlong.py pixel line infile')
    print('')
    sys.exit( 1 )

# =============================================================================

infile = None
pixel = None
line = None

# =============================================================================
# Parse command line arguments.
# =============================================================================
i = 1
while i < len(sys.argv):
    arg = sys.argv[i]

    if pixel is None:
        pixel = float(arg)

    elif line is None:
        line = float(arg)

    elif infile is None:
        infile = arg

    else:
        Usage()

    i = i + 1

if infile is None:
    Usage()
if pixel is None:
    Usage()
if line is None:
    Usage()

# Open input dataset
indataset = gdal.Open( infile, gdal.GA_ReadOnly )

# Read geotransform matrix and calculate ground coordinates
geomatrix = indataset.GetGeoTransform()
X = geomatrix[0] + geomatrix[1] * pixel + geomatrix[2] * line
Y = geomatrix[3] + geomatrix[4] * pixel + geomatrix[5] * line

# Shift to the center of the pixel
X += geomatrix[1] / 2.0
Y += geomatrix[5] / 2.0

# Build Spatial Reference object based on coordinate system, fetched from the
# opened dataset
srs = osr.SpatialReference()
if srs.ImportFromWkt(indataset.GetProjection()) != 0:
    print("ERROR: Cannot import projection '%s'" % indataset.GetProjection())
    sys.exit(1)

srsLatLong = srs.CloneGeogCS()
ct = osr.CoordinateTransformation(srs, srsLatLong)
(int, lat, height) = ct.TransformPoint(X, Y)


In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(12,12)

In [ ]:
plt.cm.Y

In [ ]:
data.shape

In [ ]:
def show_window(x=0,y=0):
    xi = slice(x*1024, (x+1)*1024)
    yi = slice(y*1024, (y+1)*1024)
    plt.imshow(data[xi, yi], cmap='gray', aspect='auto',
        interpolation='none')
    plt.grid()
    print(xi)
    print(yi)
    plt.colorbar()

In [ ]:
from IPython.html.widgets import interact

In [ ]:
interact(show_window, 
         x=(0, data.shape[0]/1024), 
         y=(0, data.shape[1]/1024));

In [ ]:
from skimage.measure import block_reduce

In [ ]:
reduced = block_reduce(data, (10,10))

In [ ]:
np.nanmean(reduced)

In [ ]:
np.nanmin(reduced)

In [ ]:
plt.imshow(reduced, cmap='gray')

In [ ]:
cropped = reduced[:150]

In [ ]:
plt.imshow(cropped)